# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

AttributeError: 'Context' object has no attribute 'param'

In [4]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1, 1, 1, 1099511623296, 1099511623296, 0, 1099511623296, 1099511623296, 0, 1099511623296, 1, 1, 0, 0, 1099511623296, 1, 1099511623296, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1099511623296, 1099511623296, 1099511623296, 0, 1099511623296, 1099511623296, 0, 1099511623296, 1099511623296, 0, 1, 1099511623296, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1099511623296, 1099511623296, 1, 1, 0, 1099511623296, 1099511623296, 0, 0, 1099511623296, 1, 1099511623296, 1, 0, 1099511623296, 1099511623296]]


In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[0, 1]     [1, 1]     [1, 0, 1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

2
3
5


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

2     3     5


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

5


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

5


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

5


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [18]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[547673600290, 182509703597, 605352823985, 535031829300, 189121239682, 199238688460, 746855160657, 626879383758, 890239667619, 136334616306, 645233577070, 379730707829, 318673394434, 652022127196, 162659296004, 1012660010564, 845916854940, 334676382180, 564977974847, 295055438742, 135775281406, 919718289214, 645953368005, 468066741347, 530071347981, 746159814619, 396435728522, 233215557072, 843228592838, 727561914241, 704247665699, 86091039060, 825021783247, 591871086487, 236737927759, 640377164113, 184186538798, 468596345651, 829937111145, 1094675881608, 883353846562, 378812248931, 1091947367842, 378331989474, 240990077467, 687053997224, 116876707805, 951688925903, 508556562347, 1057688841892, 289561684465, 1066496182842, 892656877756, 198668617796, 550796436052, 763004528125, 1064890875145, 709350343527, 908980781203, 600568704256, 13150535246, 453394347883, 537267822171, 799436989005]], [[256441165395, 826021478949, 614838782542, 273528695329, 639324189727, 856941481472, 3

In [19]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

6      6


In [20]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(128, CoeffModulus().create(128, [30]), 128))

In [21]:
keys = KeyGenerator(context).keygen()

In [22]:
x = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [23]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12fe1df70>

In [24]:
y = x.decrypt(private_key = keys[0])
y

tensor([[1, 2, 3],
        [4, 5, 6]])

In [25]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12fe1df70>

In [26]:
x = x + y

In [27]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12fe1dbb0>

In [28]:
x.decrypt(private_key = keys[0])

tensor([[ 2,  4,  6],
        [ 8, 10, 12]])

In [29]:
x = x - th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [30]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12fe4b6d0>

In [31]:
x.decrypt(private_key = keys[0])

tensor([[1, 2, 3],
        [4, 5, 6]])